# 6 讓 AI 幫 AI－自動串接流程

## 6-1 從 ChatGPT 外掛得到的啟示

### 準備工作

請依照第 2 章說明上傳 .env 檔

In [ ]:
# 設定環境變數
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

# 匯入 openai 模組
!pip install openai
import openai

# 匯入 googlesearch 模組
!pip install googlesearch-python
from googlesearch import search

# 本章範例需要從 JSON 資料轉 Python 字典
import json

### 搭配串流/非串流模式的工具函式

In [ ]:
# 可以串流或是非串流方式使用 ChatCompletion API 的函式
# 可在取得完整回覆時自動叫用 stop_func 指定的函式
def get_reply_g(messages, stream=False):
    try:
        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = messages,
            stream = stream
        )
        if stream: # 串留模式下交給輔助函式取得完整文字
            for res in response:
                if 'content' in res['choices'][0]['delta']:
                    yield res['choices'][0]['delta']['content']
        else:      # 非串流模式下可直接取得完整回覆文字
            yield response['choices'][0]['message']['content']
    except openai.OpenAIError as err:
        reply = f"發生 {err.error.type} 錯誤\n{err.error.message}"
        print(reply)
        yield reply

In [ ]:
# 測試非串流模式的情況
for reply in get_reply_g([{'role':'user', 'content':'你好'}]):
    print(reply)

In [ ]:
# 測試串流模式的情況
for msg in get_reply_g([{'role':'user', 'content':'你好'}], True):
    print(msg, end='')

## 6-2 由 AI 自動判斷要額外進行的工作

### 撰寫判斷是否需要搜尋的工具函式

In [ ]:
# 用來詢問是否需要搜尋才能回覆問題的樣板
# 要求 AI 以 JSON 格式回覆 Y/N 以及建議的搜尋關鍵字
template_google = '''
如果我想知道以下這件事, 請確認是否需要網路搜尋才做得到？

```
{}
```

如果需要, 請以下列 JSON 格式回答我, 除了 JSON 格式資料外,
不要加上額外資訊, 就算你知道答案, 也不要回覆：

```
{{
    "search":"Y",
    "keyword":"你建議的搜尋關鍵字"
}}
```
如果不需要, 請以下列 JSON 格式回答我：

```
{{
    "search":"N",
    "keyword":""
}}
'''

In [ ]:
# 利用目前歷史紀錄以及樣板內容詢問是否需要搜尋才能回覆問題
# 如果需要回覆, 也同時取得 AI 推薦的搜尋關鍵字
def check_google(hist, msg, verbose=False):
  reply = get_reply_g(
    hist + [{  # 加入歷史紀錄 AI 才能推薦正確的關鍵字
    "role": "user",
    "content": template_google.format(msg)
  }])
  for ans in reply:pass
  if verbose: print(ans)
  return ans

In [ ]:
# 測試需要搜尋的狀況
ans = check_google(
    [], '2023 NBA 冠軍是哪一隊？', True
)
# 測試可能不需要搜尋的狀況
ans = check_google(
    [], '新冠疫情是哪一年開始的？', True
)
# 測試沒有前文脈絡的狀況
ans = check_google(
    [], '那台灣呢？', True
)
# 測試包含前文脈絡的狀況
ans = check_google(
    [{'role':'assistant', 'content': '印度空污好嚴重'}],
    '那台灣呢？', True
)

In [ ]:
def google_res(user_msg, num_results=5, verbose=False):
    content = "以下為已發生的事實：\n"                # 強調資料可信度
    for res in search(user_msg, advanced=True,    # 一一串接搜尋結果
                      num_results=num_results,
                      lang='zh-TW'):
        content += f"標題：{res.title}\n" \
                    f"摘要：{res.description}\n\n"
    content += "請依照上述事實回答以下問題：\n"        # 下達明確指令
    if verbose:
        print('------------')
        print(content)
        print('------------')
    return content

In [ ]:
res = google_res('2023 NBA 冠軍隊', 2, verbose=True)

### 可自行判斷是否進行網路搜尋的聊天程式

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_g(sys_msg, user_msg, stream=False, verbose=False):
    global hist
    messages = [{'role':'user', 'content':user_msg}]
    ans = json.loads(check_google(hist, user_msg,
                                  verbose=verbose))
    if ans['search'] == 'Y':
        print(f'嘗試透過網路搜尋：{ans["keyword"]}....')
        res = google_res(ans['keyword'], verbose=verbose)
        messages = [{'role':'user', 'content': res + user_msg}]

    reply_full = ''
    replies = get_reply_g(            # 使用搜尋版的函式
        hist        # 先提供歷史紀錄
        + messages  # 再提供搜尋結果及目前訊息
        + [{"role": "system", "content": sys_msg}],
        stream)
    for reply in replies:
        reply_full += reply
        yield reply

    hist += [
        {"role":"user", "content":user_msg},
        {"role":"assistant", "content":reply_full}
    ]
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)                   # 移除最舊的紀錄

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()

while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    # 不論是字串或是生成器, 都可以適用 for...in 迴圈
    for reply in chat_g(sys_msg, msg, stream=False):
        print(reply, end = "")
    print('\n')
hist = []

## 6-3 可建構外掛系統的 Function Calling 機制

**Function calling 機制**

Function calling 機制可以讓我們提供可用函式的規格, 由 AI 幫我們判斷是否需要叫用其中的函式。

### 告知語言模型可用的外部工具函式

In [ ]:
response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [{
        "role":"user", "content":"2023 金曲歌后？"
    }],
    functions = [{                                # 可用的函式清單
        "name": "google_res",                     # 函式名稱
        "description": "取得 Google 搜尋結果",      # 函式說明
        "parameters": {
            "type": "object",
            "properties": {
                "user_msg": {                     # 參數名稱
                    "type": "string",             # 資料類型
                    "description": "要搜尋的關鍵字", # 參數說明
                }
            },
            "required": ["user_msg"],             # 必要參數
        },
    }],
    function_call = "auto",        # 請 AI 判斷是否需要叫用函式
)

若 API 判斷需要叫用你描述的函式, 會在回覆中以 function_call 項目描述要叫用的函式名稱與參數值。

### 取得語言模型的建議

In [ ]:
print(response)

In [ ]:
func_call = response['choices'][0]['message']['function_call']
func_name = func_call["name"]
args = json.loads(func_call["arguments"])
arg_val = args.popitem()[1]
print(f'{func_name}("{arg_val}")')

### 執行函式並傳回結果

你必須自行叫用函式, 並且將執行結果透過 function 角色的訊息傳回

In [ ]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"user", "content":"2023 金曲歌后？"},
        # 傳回 AI 傳給我們的 function calling 結果
        response["choices"][0]["message"],
        {   # 以 function 角色加上 name 屬性指定函式名稱傳回執行結果
            "role": "function",
            "name": func_name,
            "content": eval(f'{func_name}("{arg_val}")')
        }
    ]
)

In [ ]:
print(response['choices'][0]['message']['content'])

### 以串流方式使用 function calling

In [ ]:
response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [{
        "role":"user", "content":"2023 金曲歌后？"
    }],
    functions = [{                                # 可用的函式清單
        "name": "google_res",                     # 函式名稱
        "description": "取得 Google 搜尋結果",      # 函式說明
        "parameters": {
            "type": "object",
            "properties": {
                "user_msg": {                     # 參數名稱
                    "type": "string",
                    "description": "要搜尋的關鍵字", # 參數說明
                }
            },
            "required": ["user_msg"],
        },
    }],
    function_call = "auto", # 請 AI 判斷是否需要叫用函式
    stream=True
)

傳回結果一樣是生成器

In [ ]:
for chunk in response:
    print(chunk)

## 6-4 建立 API 外掛系統

### 建立外部工具函式參考表

建立以 function calling 為基礎的外掛機制。<br>
建立結構化的函式表格。

In [ ]:
func_table = [
    {                       # 每個元素代表一個函式
        "chain": True,      # 函式執行結果是否要再傳回給 API
        "func": google_res, # 函式
        "spec": {           # function calling 需要的函式規格
            "name": "google_res",
            "description": "取得 Google 搜尋結果",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_msg": {
                        "type": "string",
                        "description": "要搜尋的關鍵字",
                    }
                },
                "required": ["user_msg"],
            },
        }
    }
]

### 建立協助 function calling 的工具函式
依據回應內容自動叫用對應函式：

In [ ]:
# 從 API 傳回的 function_calling 物件中
# 取出函式名稱與參數內容自動呼叫函式並傳回結果
def call_func(func_call):
    func_name = func_call['name']
    args = json.loads(func_call['arguments'])
    for f in func_table: # 找出包含此函式的項目
        if func_name == f['spec']['name']:
            print(f"嘗試叫用：{func_name}(**{args})")
            val = f['func'](**args)
            return val, f['chain']
    return '', False

In [ ]:
# 從 API 傳回內容找出 function_calling 內容
def get_func_call(messages, stream=False, func_table=None,
                  **kwargs):
    model = 'gpt-3.5-turbo'
    if 'model' in kwargs: model = kwargs['model']
    funcs = {}
    if func_table:
        funcs = {'functions':[f['spec'] for f in func_table]}
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        stream = stream,
        **funcs
    )
    if stream:
        chunk = next(response)
        delta = chunk["choices"][0]["delta"]
        if 'function_call' in delta:
            func_call = delta['function_call']
            args = ''
            for chunk in response:
                delta = chunk["choices"][0]["delta"]
                if 'function_call' in delta:
                    args += delta['function_call']['arguments']
            func_call['arguments'] = args
            return func_call, None
    else:
        msg = response["choices"][0]["message"]
        if 'function_call' in msg:
            return msg['function_call'], None
    return None, response

### 建立 function_calling 版的 get_reply_f() 函式

In [ ]:
def get_reply_f(messages, stream=False, func_table=None,
                **kwargs):
    try:
        func_call, response = get_func_call(messages,
                                            stream, func_table,
                                            **kwargs)
        if func_call:
            res, chain = call_func(func_call)
            if chain:  # 如果需要將函式執行結果送回給 AI 再回覆
                messages += [
                    {  # 必須傳回原本 function_calling 的內容
                        "role": "assistant",
                        "content": None,
                        "function_call": func_call
                    },
                    {  # 以及以 function 角色的函式執行結果
                        "role": "function",        # function 角色
                        "name": func_call['name'], # 傳回函式名名稱
                        "content": res             # 傳回執行結果
                    }]
                yield from get_reply_f(messages, stream,
                                       func_table, **kwargs)
            else:
                yield res
        elif stream:
            for chunk in response:
                if 'content' in chunk['choices'][0]['delta']:
                    yield chunk['choices'][0]['delta']['content']
        else:
            yield response['choices'][0]['message']['content']
    except openai.OpenAIError as err:
        reply = f"發生 {err.error.type} 錯誤\n{err.error.message}"
        print(reply)
        yield reply

In [ ]:
# 測試非串流方式 function_calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金曲歌后是誰？"}],
    func_table=func_table):
    print(chunk)

In [ ]:
# 測試串流方式 function_calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金曲歌后是誰？"}],
    stream=True,
    func_table=func_table):
    print(chunk, end='')

In [ ]:
# 測試非串流、無 function calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金曲歌后是誰？"}]):
    print(chunk)

In [ ]:
# 測試串流、無 function calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金曲歌后是誰？"}],
    stream=True):
    print(chunk, end='')

### 建立 function calling 版本的 chat_f() 函式

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_f(sys_msg, user_msg, stream=False, **kwargs):
    global hist

    replies = get_reply_f(    # 使用函式功能版的函式
        hist                          # 先提供歷史紀錄
        + [{"role": "user", "content": user_msg}]
        + [{"role": "system", "content": sys_msg}],
        stream, func_table, **kwargs)
    reply_full = ''
    for reply in replies:
        reply_full += reply
        yield reply

    hist += [
        {"role":"user", "content":user_msg},
        {"role":"assistant", "content":reply_full}
    ]
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)  # 移除最舊的紀錄

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_f(sys_msg, msg, stream=True):
        print(reply, end = "")
    print('\n')
hist = []

## 6-5 迭代式 function-calling 機制

### gpt-4 模型真的比較厲害

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_f(sys_msg, msg, stream=True, model='gpt-4'):
        print(reply, end = "")
    print('\n')
hist = []

### 驗證答案必要時強制進行第二輪

In [ ]:
verify_msg = {
    "role": "user",
    "content": "如果之前是問句, 且剛剛的回答內容確實已經回答了問題, "
               "請回覆 'Y', 否則回覆 'N', 只要單一字母,"
               " 不要加任何其他說明"
}

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_v(sys_msg, user_msg, stream=False, **kwargs):
    global hist
    verify = kwargs.get('verify', False)
    while True:
        replies = get_reply_f(    # 使用函式功能版的函式
            hist                          # 先提供歷史紀錄
            + [{"role": "user", "content": user_msg}]
            + [{"role": "system", "content": sys_msg}],
            stream, func_table, **kwargs)
        reply_full = ''
        for reply in replies:
            reply_full += reply
            yield reply

        hist += [
            {"role":"user", "content":user_msg},
            {"role":"assistant", "content":reply_full}
        ]
        if not verify: break
        for reply in get_reply_f(hist + [verify_msg]):pass
        print(f"(已完成：{reply})")
        if reply=='Y': break
        user_msg = '繼續'
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)  # 移除最舊的紀錄

In [ ]:
hist = []
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_v(
        sys_msg, msg, stream=True,
        model='gpt-4',
        verify=True,
        debug=True
        ):
        print(reply, end = "")
    print('\n')